In [11]:
import yfinance as yf
import pandas as pd
import statsmodels.api as sm
import warnings
warnings.filterwarnings("ignore")

In [15]:
stocks = ["AAPL", "MSFT", "GOOGL", "AMZN", "TSLA", "META", "NVDA", "JPM", "V", "JNJ", 'VOO', 'VOOG', 'VGT']
market = yf.download("^GSPC", start="2020-01-01", end="2025-09-01", progress=False)['Close']
market_ret = market.pct_change().dropna()
rf_daily = 0.04 / 252

results = []

for ticker in stocks:
    stock = yf.download(ticker, start="2020-01-01", end="2025-01-01", progress=False)['Close']
    stock_ret = stock.pct_change().dropna()
    # Align indexes
    data = pd.concat([stock_ret, market_ret], axis=1, join="inner")
    data.columns = ["Stock", "Market"]
    data["Stock_excess"] = data["Stock"] - rf_daily
    data["Market_excess"] = data["Market"] - rf_daily
    X = sm.add_constant(data["Market_excess"])
    y = data["Stock_excess"]
    model = sm.OLS(y, X).fit()
    alpha, beta = model.params
    market_premium = data["Market"].mean() * 252 - 0.04  # annualized
    expected_return = 0.04 + beta * market_premium
    results.append((ticker, alpha, beta, expected_return))

for ticker, alpha, beta, expected_return in results:
    print(f"{ticker}: Alpha={alpha:.6f}, Beta={beta:.4f}, Expected annual return (CAPM)={expected_return:.2%}")

AAPL: Alpha=0.000552, Beta=1.1733, Expected annual return (CAPM)=15.89%
MSFT: Alpha=0.000357, Beta=1.1741, Expected annual return (CAPM)=15.90%
GOOGL: Alpha=0.000408, Beta=1.1328, Expected annual return (CAPM)=15.48%
AMZN: Alpha=0.000330, Beta=1.0811, Expected annual return (CAPM)=14.96%
TSLA: Alpha=0.002204, Beta=1.5778, Expected annual return (CAPM)=19.99%
META: Alpha=0.000540, Beta=1.3081, Expected annual return (CAPM)=17.26%
NVDA: Alpha=0.002184, Beta=1.7577, Expected annual return (CAPM)=21.81%
JPM: Alpha=0.000153, Beta=1.0805, Expected annual return (CAPM)=14.95%
V: Alpha=0.000015, Beta=1.0158, Expected annual return (CAPM)=14.29%
JNJ: Alpha=-0.000173, Beta=0.4826, Expected annual return (CAPM)=8.89%
VOO: Alpha=0.000063, Beta=0.9972, Expected annual return (CAPM)=14.11%
VOOG: Alpha=0.000130, Beta=1.0982, Expected annual return (CAPM)=15.13%
VGT: Alpha=0.000264, Beta=1.2240, Expected annual return (CAPM)=16.40%
